<a href="https://colab.research.google.com/github/valsson-group/UNT-ChemicalApplicationsOfMachineLearning-Spring2026/blob/main/Lecture-7_February-10-2026/Assignment-2_SomeComments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chemical Applications of Machine Learning (CHEM 4930/5610) - Spring 2026

### Assignment 2 - Deadline 2/3/2026
Points 10

#### General Comments
All figures and graph should have approriate labels on the two axis, and should include a legend with appropriate labels of the different plots.

The notebook should be return in working format. That is, I should be able to reset all the output and re-run all the cells and get the same results as you obtained.

**You should start by saving a copy of the notebook to your Google Drive so you preserve all changes.**

**Please add your name as a suffix to the filname**

**Student Name**: Add your name here

**AI usage statement:**
Here you should give a statement about any usage of AI tools to assist you with the coding.

### Task 1 - 10 points

In this task, we will consider the Bradley Melting Point Dataset, which is curated chemical dataset with melting points of around 3,000 chemical compounds, see [here](https://www.kaggle.com/datasets/aliffaagnur/melting-point-chemical-dataset/data).

This dataset is stored in a comma-separated values (csv) file, which is common format used to start data in text files. We can load this into a pandas DataFrame using the `load_csv` function.

In this dataset, we have the compounds names, SMILES strings, and the melting point in Celsius.

#### A)
Identify in the dataset the chemical compounds with the 5 lowest melting points and 5 highest melting points and visualize their 2D chemical structure using RDKit and the [mols2grid package](https://mols2grid.readthedocs.io/en/latest/), where you display the melting point values on the grid, see [here](https://colab.research.google.com/github/PatWalters/practical_cheminformatics_tutorials/blob/main/fundamentals/A_Whirlwind_Introduction_To_The_RDKit.ipynb#scrollTo=N3CR7rMF3sg7) for an example of the usage of mols2grid.

#### B)
Calculate the following properties for the molecules using RDKIt:
- The molecular weight
- The number of heavy atoms
- Number of hydrogen bond acceptors
- Number of hydrogen bond donors
- [Octanol-water partition coefficient - LogP](https://pubs-acs-org.libproxy.library.unt.edu/doi/10.1021/ci990307l)
- [Topological polar surface area (TPSA) descriptor](https://pubs-acs-org.libproxy.library.unt.edu/doi/abs/10.1021/jm000942e)
- Topological polar surface area (TPSA) descriptor, including S and P atoms, see [here](https://www.rdkit.org/docs/RDKit_Book.html#implementation-of-the-tpsa-descriptor)

Note: for some of the molecules, the TPSA descriptor will give a value of zero. When doing any analysis for the TPSA descriptor, you should ignore these values.

#### C)
Write out to a new csv file values of all the properties calculated in B) along with the compound names, SMILES strings, and the melting point in Celsius. Here, when writing this file, you should ignore any compounds where the SMILES conversion did not work correctly.

#### D)
Perform a linear regression analysis using scikit-learn where you look at the correlation of each of the properties calculated in B) with melting temperature. Here, each property should be considered individually.

To avoid outliers, filter out (i.e., remove) the compounds with the lowest 10% and the highest 10% melting temperature. Make a histogram that shows this filtering. Furthermore, for each property, filter out the compounds with lowest 10% and highest 10% values (again making a histogram that shows this filtering). Only consider the joint remaining compounds in your linear regression analysis for each property.

When performing the linear regression, employ a 70%/30% training/test split.

Calculate the coefficient of determination, $R^2$, for both the training dataset and the test dataset and report both.

You should make figure that shows the data along with the linear curve coming from the linear regression. In the figure, it should be clear which data points are in the training and test set (e.g., by having them in different colors). Include the $R^2$ values on the figure.

From your analysis, which of the properties correlates best with the melting temperature?

#### E)
For two of the properties from D) (e.g., the ones that correlate best with the melting point), perform [RANSAC](https://en.wikipedia.org/wiki/Random_sample_consensus) regression, which is method that takes outliers into account when performing linear regression and does not include them in the final modeling, see [here](https://scikit-learn.org/stable/auto_examples/linear_model/plot_ransac.html).

In the figure, it should be clear which data points are in inlier set and which are in the outlier set (e.g., by showing them in different colors).


In [2]:
# Bash script to download all the dataset. Don't worry if you don't understand it
%%bash

url="https://raw.githubusercontent.com/valsson-group/UNT-ChemicalApplicationsOfMachineLearning-Spring2026/refs/heads/main/Assignment-2/"
dataset_filename="BradleyDoublePlusGoodMeltingPointDataset.csv"

rm -f ${dataset_filename}

wget ${url}/${dataset_filename} &> /dev/null

ls

BradleyDoublePlusGoodMeltingPointDataset.csv
sample_data


In [3]:
data_mp

NameError: name 'data_mp' is not defined

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
# the %%capture command will surpress output to screen
%%capture
!pip install rdkit mols2grid

In [13]:
from rdkit import Chem

In [6]:
data_mp = pd.read_csv("BradleyDoublePlusGoodMeltingPointDataset.csv")


In [7]:
data_mp

,key,name,smiles,mpC,csid,link,source,count,min,max,range
0,27956,cyclobutylmethane,C1(CCC1)C,-161.51,11232,http://pubs.acs.org/doi/abs/10.1021/ja01142a048,Lemaire HP; Livingston RL Journal of the Ameri...,2,-161.51,-161.5,0.01
1,16005,Nitrogen oxide,[O-][N+]#N,-90.80,923,http://msds.chem.ox.ac.uk/,academic website,2,-90.81,-90.8,0.01
2,16127,Sulfuryl difluoride,FS(F)(=O)=O,-135.80,16647,http://msds.chem.ox.ac.uk/,academic website,2,-135.82,-135.8,0.02
3,17138,disopyramide,CC(C)N(CCC(c1ccccn1)(c2ccccc2)C(N)=O)C(C)C,94.80,3002,http://dx.doi.org/10.1021/ci700307p,Hughes LD; Palmer DS; Nigsch F and Mitchell JB...,2,94.75,94.8,0.05
4,15628,Bromine,BrBr,-7.20,22817,http://msds.chem.ox.ac.uk/,academic website,2,-7.25,-7.2,0.05
...,...,...,...,...,...,...,...,...,...,...,...
3036,27698,4-Nitrobenzoic acid,C1=CC(=CC=C1C(=O)O)[N+](=O)[O-],240.00,5882,http://dx.doi.org/10.1016/j.chemosphere.2013.1...,Abraham M.H. and Acree Jr. W.E. The solubility...,6,237.00,242.0,5.00
3037,28584,Thalidomide,C1CC(=O)NC(=O)C1N2C(=O)C3=CC=CC=C3C2=O,275.00,5233,http://dx.doi.org/10.1016/j.chemosphere.2013.1...,Abraham M.H. and Acree Jr. W.E. The solubility...,7,270.00,275.0,5.00
3038,28068,Estradiol,C[C@]12CC[C@H]3[C@H]([C@@H]1CC[C@@H]2O)CCC4=C3...,176.00,5554,http://dx.doi.org/10.1016/j.chemosphere.2013.1...,Abraham M.H. and Acree Jr. W.E. The solubility...,7,173.00,178.0,5.00
3039,27580,"2,4,6-Trichlorophenol",C1=C(C=C(C(=C1Cl)O)Cl)Cl,65.00,21106172,http://dx.doi.org/10.1016/j.chemosphere.2013.1...,Abraham M.H. and Acree Jr. W.E. The solubility...,9,65.00,70.0,5.00


In [8]:
# Simplify by removing columns from the data frame
data_mp = data_mp.drop(columns=['csid','link','source','count','min','max','range'])

In [9]:
data_mp

,key,name,smiles,mpC
0,27956,cyclobutylmethane,C1(CCC1)C,-161.51
1,16005,Nitrogen oxide,[O-][N+]#N,-90.80
2,16127,Sulfuryl difluoride,FS(F)(=O)=O,-135.80
3,17138,disopyramide,CC(C)N(CCC(c1ccccn1)(c2ccccc2)C(N)=O)C(C)C,94.80
4,15628,Bromine,BrBr,-7.20
...,...,...,...,...
3036,27698,4-Nitrobenzoic acid,C1=CC(=CC=C1C(=O)O)[N+](=O)[O-],240.00
3037,28584,Thalidomide,C1CC(=O)NC(=O)C1N2C(=O)C3=CC=CC=C3C2=O,275.00
3038,28068,Estradiol,C[C@]12CC[C@H]3[C@H]([C@@H]1CC[C@@H]2O)CCC4=C3...,176.00
3039,27580,"2,4,6-Trichlorophenol",C1=C(C=C(C(=C1Cl)O)Cl)Cl,65.00


In [10]:
test_none=1
if test_none:
  print("1")
else:
  print("2")

1


In [11]:
from rdkit.Chem import Descriptors

def number_of_rotatable_bonds(smi):
  mol = Chem.MolFromSmiles(smi)
  if mol:
    return Descriptors.NumRotatableBonds(mol)
  else:
    return np.nan

def molecular_weight(smi):
  mol = Chem.MolFromSmiles(smi)
  if mol:
    return Descriptors.MolWt(mol)
  else:
    return np.nan

In [14]:
data_mp['NumRotatableBond'] = [number_of_rotatable_bonds(smi) for smi in data_mp['smiles']]
data_mp['MW'] = [molecular_weight(smi) for smi in data_mp['smiles']]

[17:26:29] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[17:26:29] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6
[17:26:29] Can't kekulize mol.  Unkekulized atoms: 24 25 26 27 28 31 32 33 34
[17:26:29] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4
[17:26:29] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 8
[17:26:29] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[17:26:29] Can't kekulize mol.  Unkekulized atoms: 16 17 18 19 20 21 22 23 24
[17:26:29] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7 8 9 10 11
[17:26:29] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8
[17:26:30] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 8
[17:26:30] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[17:26:30] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 8
[17:26:30] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 12 13 14 15 16
[17:26:30] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 8
[17:26:30] Can't kekulize mol.  Unkekuliz

In [15]:
data_mp.describe()

,key,mpC,NumRotatableBond,MW
count,3041.000000,3041.000000,3025.000000,3025.000000
mean,10771.570865,62.848159,2.296860,180.528716
std,6875.518120,96.007422,3.439933,80.114001
min,10.000000,-188.000000,0.000000,16.043000
25%,5148.000000,5.000000,0.000000,129.247000
50%,13103.000000,64.000000,1.000000,166.180000
75%,15269.000000,129.500000,3.000000,214.648000
max,28644.000000,438.000000,47.000000,959.171000


In [16]:
mw_p10 = np.percentile(data_mp['MW'],10)
print(mw_p10)

nan


In [17]:
mw_p10 = np.nanpercentile(data_mp['MW'],10)
print(mw_p10)

98.18899999999998


In [18]:
data_mp

,key,name,smiles,mpC,NumRotatableBond,MW
0,27956,cyclobutylmethane,C1(CCC1)C,-161.51,0.0,70.135
1,16005,Nitrogen oxide,[O-][N+]#N,-90.80,0.0,44.013
2,16127,Sulfuryl difluoride,FS(F)(=O)=O,-135.80,0.0,102.061
3,17138,disopyramide,CC(C)N(CCC(c1ccccn1)(c2ccccc2)C(N)=O)C(C)C,94.80,8.0,339.483
4,15628,Bromine,BrBr,-7.20,0.0,159.808
...,...,...,...,...,...,...
3036,27698,4-Nitrobenzoic acid,C1=CC(=CC=C1C(=O)O)[N+](=O)[O-],240.00,2.0,167.120
3037,28584,Thalidomide,C1CC(=O)NC(=O)C1N2C(=O)C3=CC=CC=C3C2=O,275.00,1.0,258.233
3038,28068,Estradiol,C[C@]12CC[C@H]3[C@H]([C@@H]1CC[C@@H]2O)CCC4=C3...,176.00,0.0,272.388
3039,27580,"2,4,6-Trichlorophenol",C1=C(C=C(C(=C1Cl)O)Cl)Cl,65.00,0.0,197.448


In [19]:
# to remove anything that is NaN
data_mp = data_mp.dropna(axis=0, subset=['MW'])

In [20]:
data_mp

,key,name,smiles,mpC,NumRotatableBond,MW
0,27956,cyclobutylmethane,C1(CCC1)C,-161.51,0.0,70.135
1,16005,Nitrogen oxide,[O-][N+]#N,-90.80,0.0,44.013
2,16127,Sulfuryl difluoride,FS(F)(=O)=O,-135.80,0.0,102.061
3,17138,disopyramide,CC(C)N(CCC(c1ccccn1)(c2ccccc2)C(N)=O)C(C)C,94.80,8.0,339.483
4,15628,Bromine,BrBr,-7.20,0.0,159.808
...,...,...,...,...,...,...
3036,27698,4-Nitrobenzoic acid,C1=CC(=CC=C1C(=O)O)[N+](=O)[O-],240.00,2.0,167.120
3037,28584,Thalidomide,C1CC(=O)NC(=O)C1N2C(=O)C3=CC=CC=C3C2=O,275.00,1.0,258.233
3038,28068,Estradiol,C[C@]12CC[C@H]3[C@H]([C@@H]1CC[C@@H]2O)CCC4=C3...,176.00,0.0,272.388
3039,27580,"2,4,6-Trichlorophenol",C1=C(C=C(C(=C1Cl)O)Cl)Cl,65.00,0.0,197.448


In [21]:
mw_p10 = np.percentile(data_mp['MW'],10)
print(mw_p10)

98.18899999999998


In [22]:
data_mp.describe()

,key,mpC,NumRotatableBond,MW
count,3025.000000,3025.000000,3025.000000,3025.000000
mean,10746.770248,62.254496,2.296860,180.528716
std,6885.046638,95.663463,3.439933,80.114001
min,10.000000,-188.000000,0.000000,16.043000
25%,5112.000000,5.000000,0.000000,129.247000
50%,12815.000000,63.000000,1.000000,166.180000
75%,15264.000000,129.000000,3.000000,214.648000
max,28644.000000,438.000000,47.000000,959.171000


In [29]:
my_var=2



In [33]:
mw = data_mp['MW'].to_numpy()
print(mw.shape)

(3025,)


### Task 2 - Optional 5 points

Here we will consider a dataset of two variables $x$ and $y$ sampled from a two-dimensional probability density $P(x,y)$ that is unknown.

The dataset is given as a time series in the file `Dataset_RotatedWQ-Potential.data`.

The main task is to perform a Gaussian Mixture Model analysis on this two-dimensional dataset.

#### A)
Plot the dataset, both the time series and also a scatter plot for the $x$ and $y$ variables.

Looking at the scatter plot, how many Gaussian components do you think are needed in the Gaussian Mixture Model analysis?

#### B)
Using Seaborn (or scikit-learn) estimate the two-dimensional probability density $P(x,y)$ using kernel density estimation.

#### C)
Perform a Gaussian Mixture Model analysis for a different number of components, and obtain the Bayesian information criterion (bic) and Akaike information criterion (aic) values and based on them identify the optimal number of components (remember that for both a lower value is better).

#### D)
For the optimal number of components, perform a final Gaussian Mixture Model analysis that you will analyze.

- What is the weight of each Gaussian components.

- What is the percentage of samples that are hard classifed to each cluster.

- Make a scatter plot that shows how the samples are hard classifed to each cluster. In this plot, indicate the center of each Gaussian components.

- Make figures that shows how the samples are soft classifed to each cluster (e.g., the probablity that they belong to a given cluster). In each plot, indicate the center of corresponding Gaussian components.

- Plot a two-dimensional surface of the $P(x,y)$ estimated by the Gaussian Mixture Model. How does this compare to the KDE plot from B)?


In [ ]:
# Bash script to download all the dataset. Don't worry if you don't understand it
%%bash

url="https://raw.githubusercontent.com/valsson-group/UNT-ChemicalApplicationsOfMachineLearning-Spring2026/refs/heads/main/Assignment-2/"
dataset_filename="Dataset_RotatedWQ-Potential.data"

rm -f ${dataset_filename}

wget ${url}/${dataset_filename} &> /dev/null

ls

